In [2]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [3]:
data = pd.read_csv('scdb-200rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [4]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

15329

In [5]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

bid_count = 0
bid_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        bid_count += 1   


print(f"#BIDs =  {bid_count}")

#BIDs =  4996


In [6]:
df_test = grouped_data.get_group(bid_ids[8])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
31393,received_tx,105,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09
31394,before_tendermint,115,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09
31395,check_tx,127,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09
31396,deliver_tx,171,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09
31397,end_block,173,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09
31398,commit_tx,217,BID,25aa014457981ec7e1332758fb18e81c6d5aa2eadbedfe...,None,1.670203e+09


In [26]:
time_all_committed_bids = {}
time_passed_bids = []
validated_time = []
accepted_time = []
committed_time = []

bid_underfit = []
bid_underfit_dict = {}

bid_overfit = []
bid_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            bid_underfit.append(temp_list)
            bid_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            #time_all_committed_bids[tx_id[i]] = actual_time
            #time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            bid_overfit.append(actual_time)
            bid_overfit_dict[tx_id[i]] = temp_list

In [30]:
df = grouped_data.get_group("3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336")
df[df['Event'] == 'received_tx']['Time'].to_list()[0]

106

In [28]:
len(time_all_committed_bids.values())

4451

In [29]:
len(time_passed_bids)

4451

In [30]:
len(time_passed_bids)

4451

In [31]:
len(bid_overfit)

514

In [32]:
stat.mean(time_passed_bids)

135.6209840485284

In [33]:
stat.mean(bid_overfit)

3439.3910505836575

In [34]:
# Analysis of CREATE Transaction
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))

print(bid_mean)
print(bid_median)
print(bid_max)
print(bid_min)
print(bid_std_dev)
print(f"total bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : len(time_passed_bids)}

135.62
86
7187
74
428.50
total bids = 4451


In [35]:
results = pd.DataFrame(bid_dict, index = ['BID'])
results.to_csv('bid_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
BID,74,7187,135.62,86,428.50,4451
